In [1]:
import os
from dotenv import load_dotenv

import textwrap


#This is optional. I use VPN in my computer. Why I need this. 
import truststore
truststore.inject_into_ssl()



def pretty_print(*args):
    text = " ".join(str(arg) for arg in args)
    try:
        print(textwrap.fill(text, width=80))
    except Exception as e:
        print(text)  # fallback to normal print if text is not a string

        

load_dotenv('/Users/shivam13juna/Documents/scaler/iitr_classes/jan_2026/language_model_api_v2/openai_key.env')  # reads .env file in the current directory

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found! "
        "Make sure you have a .env file with: OPENAI_API_KEY=sk-..."
    )

pretty_print("API key loaded successfully.")

API key loaded successfully.


In [2]:
from openai import OpenAI

client = OpenAI(api_key=api_key)
pretty_print("OpenAI client ready.")

OpenAI client ready.


# Responses API

## Diff b/w Chat Completions and Responses API - Structure

In [3]:

# do the same with chat completions
resp = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
		{"role": "system", "content": "You are a friendly Python tutor."},
        {"role": "user", "content": "What is a list comprehension?"}
    ]
)
pretty_print("Chat Completions output:", resp.choices[0].message.content)


Chat Completions output: A list comprehension is a concise way to create a new
list by transforming items from an existing iterable, optionally filtering them.
Basic syntax: - [expression for item in iterable] - You can add an optional
condition: [expression for item in iterable if condition]  Examples: - Squares
of numbers 0 through 9: squares = [x*x for x in range(10)] - Even numbers from 0
to 19: evens = [n for n in range(20) if n % 2 == 0] - Uppercase words: upper =
[s.upper() for s in ["apple","banana"]] - Flattening a list of lists: flat = [c
for sub in [[1,2],[3,4]] for c in sub]  Compared to a loop: - Traditional:
squares = []   for x in range(10):       squares.append(x*x) - List
comprehension does the same in one line.  Notes: - They’re great for simple
transformations and filters, but can hurt readability if too complex. - They
create a list in memory. If you want lazy evaluation, use a generator
expression: (expression for item in iterable if condition) - You can also use
d

In [4]:
resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a friendly Python tutor.",
    input="What is a list comprehension?"
)

pretty_print("Responses API output:", resp.output_text)


Responses API output: A list comprehension is a compact way to create a new list
by applying an expression to each item in an iterable, optionally filtering
items or nesting loops.  Common form: - Basic: [expression for item in iterable]
- With a filter: [expression for item in iterable if condition] - With multiple
loops: [expression for a in A for b in B]  Examples: - Squares: [x*x for x in
range(10)] - Evens from 0–19: [x for x in range(20) if x % 2 == 0] - Pairs: [(i,
j) for i in range(3) for j in range(2)]  Notes: - It’s equivalent to building a
list with a for-loop, but shorter and often more readable. - If you don’t want
to build a list in memory, you can use a generator expression: (expression for
item in iterable).


## Passing Multi Turn Conversation History to Responses API

In [5]:
input_messages = [
    {"role": "user", "content": "Why is Trump a jerk?"},
    {"role": "assistant", "content": "Some people are born that way."},
    {"role": "user", "content": "Name a celebrity who is not a jerk."}
]


resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a very candid Journalist.",
    input=input_messages
)

pretty_print("Responses API output:", resp.output_text)

Responses API output: Subjectively, many people point to these as not jerks:  -
Keanu Reeves — widely described as gracious, humble, and generous. - Tom Hanks —
known for warmth and decency in public appearances. - Dwayne “The Rock” Johnson
— often praised for positivity and philanthropy.  Of course, public personas
vary by viewer, but these are commonly viewed as pleasant, not jerky, figures.
If you want someone from a specific field (music, sports, etc.), I can tailor
suggestions.


| Feature             | Chat Completions API                             | Responses API                                              |
| ------------------- | ------------------------------------------------ | ---------------------------------------------------------- |
| **Endpoint**        | `client.chat.completions.create()`               | `client.responses.create()`                                |
| **Input format**    | `messages=[{"role": ..., "content": ...}]`       | `input=` (string or list of message dicts)                 |
| **System prompt**   | `{"role": "system", "content": ...}` in messages | `instructions=` parameter (top-level)                      |
| **Output access**   | `resp.choices[0].message.content`                | `resp.output_text`                                         |
| **Multi-turn**      | Manually pass full message history each time     | `previous_response_id=resp.id` (server-side context)       |
| **Developer role**  | Not supported (use `system`)                     | `{"role": "developer"}` for meta-instructions              |
| **Vision input**    | `{"type": "image_url", "image_url": {...}}`      | `{"type": "input_image", "image_url": ...}`                |
| **Reasoning / CoT** | Not natively supported                           | `reasoning={"effort": ..., "summary": ...}` built-in       |
| **Response object** | `ChatCompletion` with `choices[]` list           | `Response` with `output[]` list and `output_text` shortcut |
| **Streaming**       | `stream=True` yields `ChatCompletionChunk`       | `stream=True` yields server-sent events                    |
| **Tool calls**      | Supported via `tools` param                      | Supported via `tools` param (same)                         |
| **Model support**   | All chat models                                  | All chat models (newer, recommended going forward)         |


In [6]:
input_messages = [
    {"role": "user", "content": "Why is Trump a jerk?"},
    {"role": "assistant", "content": "Some people are born that way."},
    {"role": "user", "content": "Name a celebrity who is not a jerk."}
]


resp = client.responses.create(
    model="gpt-5-nano",
    instructions="You are a very candid Journalist.",
    input=input_messages, 
    reasoning={"effort": "minimal"}
)

pretty_print("Responses API output:", resp.output_text)

Responses API output: That’s a tricky question—“not a jerk” is pretty subjective
and can flip with the camera on. If you’re asking for someone widely regarded as
gracious or down-to-earth in public life, a few names people frequently bring up
are Tom Hanks or Keanu Reeves. They’re often cited for kindness and humility,
though there’s always room for different perspectives. Want me to pull a few
examples with receipts (quotes, interviews, philanthropy) to back it up?


link to Documentation

[Chat Completions](https://developers.openai.com/api/reference/python/resources/chat/subresources/completions/methods/create)

[Responses API](https://developers.openai.com/api/reference/python/resources/responses/methods/create)